## NOTEBOOK DESCRIPTION:

This notebook creates a sparse matrix which represent a proximity graph.
Since the dataset is sorted according to each user we sequentially consider each user: 

	For each user:
		We select the channels this user commented in.
        And then, since our comments don’t have a time reference when it was posted we decided to construct edges of the graph by doing all the possible combinations out of the selected channels.

In [1]:
import json
import time
import pickle
import scipy.sparse
import sys
import os
import random
import itertools
import math
import glob

import zstandard as zstd
import pandas as pd
import numpy as np

from scipy.sparse import dok_matrix

scriptpath = "/home/jouven/youtube_projects/"
sys.path.append(os.path.abspath(scriptpath))

from helpers.helpers_channels_more_300 import *

In [4]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = video_id_to_channel_id()

In [2]:
# Channels with the selected comments
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

In [6]:
# Set of duplicate users
duplicate_users = dict_occurent_users()

In the goal of evaluating the the similarity between channels we created what we call a proximity graph.
As the dataset is sorted according to each user we sequentially consider each user:
For each user:
	We select the channels this user commented in.
		
And then, since our comments don’t have a time reference when it was posted we decided to construct edges of the graph by doing all the possible combinations out of the selected channels.

In [8]:
'''
Function to add new edges to the existing data. Edges are formed by doing the 2-combinations of user_channels. 
If the edge already exists -> add 1 to the existing weight
    PARAMETERS:
        - graph_dict: dictionnary mapping the edge (tuple of channel indices) with the weight of that edge
        - user_channels: set of channels that a user commented in
'''

def create_edges(graph_dict, user_channels):
    
    for comb in itertools.combinations(user_channels, 2):
        edge = (comb[0], comb[1])
        if edge in graph_dict:
            graph_dict[edge] += 1
        else:
            graph_dict[edge] = 1
            
'''
Function to add new edges to the existing data. Edges are formed by first selecting the context channels at random
and them by doing the 2-combinations of user_channels. 
If the edge already exists -> add 1 to the existing weight
    PARAMETERS:
        - graph_dict: dictionnary mapping the edge (tuple of channel indices) with the weight of that edge
        - user_channels: set of channels that a user commented in
        - context: maximum number channels selected for a given user
'''
def create_edges_limited(graph_dict, user_channels, context):
    
    if len(user_channels) > context:
        user_edge = random.sample(user_channels, context)
    
    for comb in itertools.combinations(user_channels, 2):
        edge = (comb[0], comb[1])

        if edge in graph_dict:
            graph_dict[edge] += 1
        else:
            graph_dict[edge] = 1
        
            

In [9]:
'''
Function to add new edges to the existing data. Edges are formed by doing the 2-combinations of user_channels. 
If the edge already exists -> add 1/log(# number of channels that this user has commented in)
    PARAMETERS:
        - graph_dict: dictionnary mapping the edge (tuple of channel indices) with the weight of that edge
        - user_channels: set of channels that a user commented in
'''
def create_edges_normalized(graph_dict, user_channels):
    
    nb_comments = len(user_channels)
    
    for comb in itertools.combinations(user_channels, 2):

        edge = (comb[0], comb[1])
        if edge in graph_dict:
            graph_dict[edge] += 1/(math.log(nb_comments, 2))
        else:
            graph_dict[edge] = 1/(math.log(nb_comments, 2))

'''
Function to add new edges to the existing data. Edges are formed by first selecting the context channels at random
and them by doing the 2-combinations of user_channels. 
If the edge already exists -> add 1/log(# number of channels that this user has commented in)
    PARAMETERS:
        - graph_dict: dictionnary mapping the edge (tuple of channel indices) with the weight of that edge
        - user_channels: set of channels that a user commented in
        - context: maximum number channels selected for a given user
'''            
def create_edges_limited_normalized(graph_dict, user_channels, context):
    
    nb_comments = len(user_channels)
    if len(user_channels) > context:
        user_channels = random.sample(user_channels, context)
    
    for comb in itertools.combinations(user_edge, 2):
        edge = (comb[0], comb[1])
        if edge in graph_dict:
            graph_dict[edge] += 1/(math.log(nb_comments, 2))
        else:
            graph_dict[edge] = 1/(math.log(nb_comments, 2))

In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

# PARAMETERS

# Dictionnary counting the number of time (channel_idx, channel2_idx) appears
graph_dict_limited = {}
# Indices
nb = 1
idx = 1
# Channels that a user have commented
user_channels = []
# Number of channels, Row and columns length of the sparse matrix
matrix_len = len(channels_id)
context = 50

user = ''
begin_time = time.time()

dir_1 = '/dlabdata1/youtube_large/jouven/sparse_matrix_construction/channels_more_10k/sparse_matrices_limited_normalized/'
check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) >= 9:
        author_id = line_split[0]
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = dict_channel_ind[vid_to_channels[line_split[2]]]
            if author_id == user:
                # if user is a duplicate user
                if author_id in duplicate_users:
                    if duplicate_users[author_id] <= 1:
                        user_channels.append(corr_channel)
                else:
                    user_channels.append(corr_channel)
            else:
                if len(user_channels) >= 2:
                    create_edges_limited_normalized(graph_dict_limited, user_channels, context)
                user_channels = []
                
                if len(graph_dict_limited) >= 75000000:
                    # For space requirements every 75 millions line create a dok matrix and
                    # update it with the graph_dict dictionnary and then save it into csr format and then release memory
                    graph_matrix = dok_matrix((matrix_len, matrix_len))
                    dict.update(graph_matrix, graph_dict_limited)
                    graph_dict_limited = {}
                    # Save sparse matrix
                    scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/channels_more_10k/sparse_matrices_limited_normalized/matrice' + str(nb) + '.npz', graph_matrix.tocsr())
                    with open("/dlabdata1/youtube_large/jouven/sparse_matrix_construction/idx2.pkl",'wb') as f:
                         pickle.dump([idx], f)
                    f.close()
                    graph_matrix = []
                    nb += 1
                    print('line number: ' + str(idx) + ' time: ' + str(time.time() - begin_time))
                    begin_time = time.time()
                        
                # If user is a duplicate user
                if author_id in duplicate_users:
                    duplicate_users[author_id] += 1
                    if duplicate_users[author_id] <= 1:
                        user_channels.append(corr_channel)
                else:
                    user_channels.append(corr_channel)
           
        user = author_id
    idx += 1
    

# Store the graph sparse matrix
graph_matrix = dok_matrix((matrix_len, matrix_len))
dict.update(graph_matrix, graph_dict_limited)
graph_dict_limited = {}
# Save sparse matrix
scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/channels_more_10k/sparse_matrices_limited_normalized/matrice' + str(nb) + '.npz', graph_matrix.tocsr())

graph_matrix = []


line number: 19335121 time: 870.8132605552673
line number: 38447852 time: 909.0604336261749
line number: 57561561 time: 791.3715510368347
line number: 76668524 time: 2465.0257103443146
line number: 95589874 time: 703.9928598403931
line number: 114667081 time: 800.9108040332794
line number: 133614829 time: 760.5318920612335
line number: 152924106 time: 827.5193493366241
line number: 172189386 time: 740.7566397190094
line number: 191525277 time: 747.5453534126282
line number: 210576932 time: 810.327553987503
line number: 229862915 time: 776.4250948429108
line number: 249095720 time: 771.9502561092377
line number: 268165763 time: 756.6586303710938
line number: 287411951 time: 780.8377737998962
line number: 306650228 time: 824.6703867912292
line number: 325791874 time: 879.0189731121063
line number: 344987003 time: 780.8514273166656
line number: 364302833 time: 779.3830678462982
line number: 383304670 time: 744.7795011997223
line number: 402365506 time: 765.0383660793304
line number: 42144

In [ ]:
'''
Enable to load every sparse matrix to form only one by adding the weights of all intermediate sparse matrix
'''
matrix_len = len(channels_id)
graph = dok_matrix((matrix_len, matrix_len)).tocsr()
path = '/dlabdata1/youtube_large/jouven/sparse_matrix_construction/channels_more_10k/sparse_matrices_limited_normalized/'
files = glob.glob(path + '*.npz')
for file in files: 
    graph += scipy.sparse.load_npz(path + 'matrice' + str(i) + '.npz')
# Save the final sparse matrix
scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/final_embedding_sparse_matrix/channels_more_10k/channel_by_channel_ln_100.npz', graph)
